In [50]:
import re
from collections import Counter, defaultdict, namedtuple
from dataclasses import dataclass, field
from functools import cache
from heapq import heappop, heappush
from itertools import combinations, pairwise, permutations
from math import inf, lcm, prod, sqrt

import black
import jupyter_black
import networkx as nx
import numpy as np
from z3 import *

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def ints(text: str) -> list[int]:
    return [int(x) for x in re.findall("-?\d+", text)]


def first(iterable):
    return next(iter(iterable))


def data(day: int, parser=str, sep="\n", example=False) -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    filename = f"2023/{day}-example.txt" if example else f"2023/{day}.txt"
    sections = open(filename).read().rstrip().split(sep)
    return [parser(section) for section in sections]

In [45]:
# Day 1: Trebuchet?!
numbers = {
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9",
}


def calibration_value(line, part1=True):
    numbers_found = ""
    for start, char in enumerate(line):
        if char.isdigit():
            numbers_found += char
        if part1:
            continue
        for key, value in numbers.items():
            if line.startswith(key, start):
                numbers_found += value
    calibration_value = numbers_found[0] + numbers_found[-1]
    return int(calibration_value)


games = data(1)
points = sum([calibration_value(line) for line in games])
print(f"Part 1: {points}")  # 53921
points = sum([calibration_value(line, part1=False) for line in games])
print(f"Part 2: {points}")  # 54676

Part 1: 53921
Part 2: 54676


In [46]:
# Day 2: Cube Conundrum
games = data(2)
possible_games = 0
powers = []
for game_nr, game in enumerate(games, 1):
    max_cubes_seen = {"red": 0, "blue": 0, "green": 0}
    cubes_seen = game.split(": ")[1].split("; ")
    for cubes in cubes_seen:
        for cube in cubes.split(", "):
            count, color = cube.split()
            count = int(count)
            if count > max_cubes_seen[color]:
                max_cubes_seen[color] = count
    if (
        max_cubes_seen["red"] <= 12
        and max_cubes_seen["green"] <= 13
        and max_cubes_seen["blue"] <= 14
    ):
        possible_games += game_nr
    powers.append(prod(max_cubes_seen.values()))

print(f"Part 1: {possible_games}")  # 2563
print(f"Part 2: {sum(powers)}")  # 70768

Part 1: 2563
Part 2: 70768


In [47]:
# Day 3: Gear Ratios
def neighbors(point):
    row, col = point
    potential = (
        (row - 1, col - 1),
        (row, col - 1),
        (row + 1, col - 1),
        (row - 1, col),
        (row + 1, col),
        (row - 1, col + 1),
        (row, col + 1),
        (row + 1, col + 1),
    )
    for (r, c) in potential:
        if 0 <= r < len(expanded_rows[0]) and 0 <= c < len(expanded_rows):
            yield (r, c)


expanded_rows = data(3)
gears = defaultdict(list)
part_numbers = []

for row_number, row in enumerate(expanded_rows):
    current_number = ""
    to_check = set()
    # Add '.' to catch numbers at end of line and avoid separate check after loop
    # completion
    for col_number, char in enumerate(row + "."):
        if char.isdigit():
            current_number += char
            for point in neighbors((row_number, col_number)):
                to_check.add(point)
        elif current_number:
            for (r, c) in to_check:
                if expanded_rows[r][c] not in "0123456789.":
                    # Symbol found, we are a part_number
                    part_numbers.append(int(current_number))
                if expanded_rows[r][c] == "*":
                    # Gear found, add number connected to this gear
                    gears[(r, c)].append(int(current_number))
            current_number = ""
            to_check = set()
print(f"Part 1: {sum(part_numbers)}")  # 514969
part2 = sum((prod(gear) for gear in gears.values() if len(gear) == 2))
print(f"Part 2: {part2}")  # 78915902

Part 1: 514969
Part 2: 78915902


In [48]:
# Day 4: Scratchcards
cards = data(4)
points = 0
total_cards = Counter([*range(len(cards))])  # One of each card to begin with
for card_nr, card in enumerate(cards):
    winning_numbers, my_numbers = map(ints, card.split(": ")[1].split(" | "))
    winners = sum(1 for number in my_numbers if number in winning_numbers)
    if winners:
        points += 2 ** (winners - 1)
        total_cards.update(
            total_cards[card_nr] * [*range(card_nr + 1, card_nr + winners + 1)]
        )

print(f"Part 1: {points}")  # 28750
print(f"Part 2: {sum(total_cards.values())}")  # 10212704

Part 1: 28750
Part 2: 10212704


In [2]:
# Day 5: If You Give A Seed A Fertilizer
def get_destination(seed, mapping):
    for dest, source, range_ in zip(mapping[0::3], mapping[1::3], mapping[2::3]):
        if source <= seed <= source + range_:
            return dest - source + seed
    return seed


def get_location(seed, mappings):
    for mapping in mappings:
        seed = get_destination(seed, mapping)
    return seed


patterns = data(5, sep="\n\n")
seeds = ints(patterns[0])
mappings = [tuple(ints(chunk)) for chunk in patterns[1:]]

print(f"Part 1: {min(get_location(seed, mappings) for seed in seeds)}")

# -- Part 2 -- Yucky.
def current(low, high, dest, source, range):
    """Return two lists (lows and highs) based on a continuous range from low to high and
    a mapping and then two lists including remaining low-high pairs to check (one list
    includes lows, one includes highs)"""
    offset = dest - source
    new_low = source
    new_high = source + range - 1
    # print(f"{new_low=} {new_high=}")
    if low >= new_low and high <= new_high:
        # Range intact, return low + offset, high + offset
        # print(f"Complete overlap {offset=}")
        return [low + offset], [high + offset], [], []
    if low <= source and new_low <= high <= new_high:
        # -------low---------------------high------------
        #         |                       |
        #             |------offset-------|
        #             |                         |
        # ---------new_low-----------------new_high------
        # print(f"Partial overlap, case 1 {offset=}")
        return (
            [low, new_low + offset],
            [new_low - 1, high + offset],
            [low],
            [new_low - 1],
        )
    if low < new_low and high > new_high:
        # -------low----------------------------high-------
        #         |                              |
        #              |------offset-------|
        #              |                   |
        # ----------new_low------------new_high------------
        # print(f"Partial overlap, case 2 {offset=}")
        return (
            [low, new_low + offset, new_high + 1],
            [
                new_low - 1,
                new_high + offset,
                high,
            ],
            [low, new_high + 1],
            [new_low - 1, high],
        )
    if new_low <= low <= new_high and high > new_high:
        # -------low---------------------high-----------
        #         |                       |
        #         |------offset-------|
        #      |                      |
        # ---new_low--------------new_high--------------
        # print(f"Partial overlap, case 3 {offset=}")
        return (
            [low + offset, new_high + 1],
            [
                new_high + offset,
                high,
            ],
            [new_high + 1],
            [high],
        )
    # Non overlapping
    # print(f"No overlap")
    return [], [], [], []


def apply_mapping(low, high, mapping):
    new_lows, new_highs = [], []
    # print(f"    {low=}     {high=}")
    for dest, source, range in zip(mapping[0::3], mapping[1::3], mapping[2::3]):
        lows_remaining = [low]
        highs_remaining = [high]
        while len(lows_remaining) and len(highs_remaining):
            l, h, lows_remaining, highs_remaining = current(
                lows_remaining[0], highs_remaining[0], dest, source, range
            )
            new_lows += l
            new_highs += h
            # print(lows_remaining, highs_remaining)
    if not new_lows:
        new_lows = [low]
    if not new_highs:
        new_highs = [high]
    # print(f"{new_lows=} {new_highs=}")
    return new_lows, new_highs


seeds = ints(patterns[0])
mappings = [ints(chunk) for chunk in patterns[1:]]
input_lows = seeds[0::2]
input_highs = [low + length - 1 for low, length in zip(input_lows, seeds[1::2])]

locations = []
for lows, highs in zip(input_lows, input_highs):
    lows = [lows]
    highs = [highs]
    for mapping in mappings:
        # print("--------New mapping---------")
        # print(f"{mapping=}")
        new_lows, new_highs = [], []
        for low, high in zip(lows, highs):
            l, h = apply_mapping(low, high, mapping)
            new_lows += l
            new_highs += h
        lows = new_lows
        highs = new_highs
    # Sometimes this is zero, don't understand why. Avoid adding zeros though.
    if min(lows):
        locations.append(min(lows))

# 2520479 is the right answer, but by god this was an awful hack.
print(f"Part 2: {min(locations)}")


# # Alternative solution for part 2, running time 1m. Go backwards from potential locations
# # and see if they are one of the starting seeds.
# def get_source(location, mapping):
#     # print(f"{location=} {mapping=}")
#     for dest, source, range_ in zip(mapping[0::3], mapping[1::3], mapping[2::3]):
#         # print(f"Range: {source} - {source + range_}, offset: {source-dest}")
#         potential_location = source - dest + location
#         if source <= potential_location <= source + range_:
#             # print(f"** location={potential_location}")
#             return potential_location
#     return location


# def get_seed(location, mappings):
#     for mapping in mappings[::-1]:
#         location = get_source(location, mapping)
#     return location


# def in_range(seed, ranges):
#     for start, offset in zip(ranges[::2], ranges[1::2]):
#         if start <= seed < start + offset:
#             return True
#     return False


# for location in range(10_000_000):
#     if in_range(get_seed(location, mappings), seeds):
#         break
# print(location)

Part 1: 462648396
Part 2: 2520479


In [ ]:
# Day 6: Wait For It
times, distances = data(6, ints)

# Travel distance = speed * (time - button_press), where speed = button_press
part1 = prod(
    sum(
        1
        for button_press in range(time)
        if button_press * (time - button_press) > distance
    )
    for time, distance in zip(times, distances)
)
print(f"Part 1: {part1}")  # 512295

time = int("".join(map(str, times)))
expansion = int("".join(map(str, distances)))
part2 = sum(
    1 for button_press in range(time) if button_press * (time - button_press) > expansion
)
print(f"Part 2: {part2}")  # 36530883 (6s)

Part 1: 512295
Part 2: 36530883


In [ ]:
# Day 7: Camel Cards
def most_common_character(s):
    s = sorted(s, reverse=True)
    freq = Counter(s)
    # Sort the characters by frequency
    sorted_chars = sorted(freq, key=freq.get, reverse=True)
    return sorted_chars[0]


lines = data(7)

hands = []
for first_part in lines:
    text, bid = first_part.split()
    # Mapping A, K, Q, J, T -> e, d, c, b, a
    hand = text.translate(str.maketrans("AKQJT", "edcba"))
    if hand == "bbbbb":
        pretend_hand = "22222"
    else:
        # replace all Jokers ('b') with the most common character *excluding* b
        pretend_hand = hand.replace("b", most_common_character(hand.replace("b", "")))
    hands.append((pretend_hand, hand, int(bid)))

# Part 1
decorated = [
    (
        sorted(Counter(hand).values(), reverse=True),
        hand,
        bid,
    )
    for _, hand, bid in hands
]
decorated.sort()
print(f"Part 1: {sum(rank * item[-1] for rank, item in enumerate(decorated, 1))}")

# Part 2
decorated = [
    (
        sorted(Counter(pretend_hand).values(), reverse=True),
        hand.replace("b", "1"),  # Jokers are worth the least
        bid,
    )
    for pretend_hand, hand, bid in hands
]
decorated.sort()
# 244848487
print(f"Part 2: {sum(rank * item[-1] for rank, item in enumerate(decorated, 1))}")

Part 1: 246409899
Part 2: 244848487


In [ ]:
# Day 8: Haunted Wasteland
patterns = data(8, sep="\n\n")
instructions = [int(step.replace("R", "1").replace("L", "0")) for step in patterns[0]]
hops = {
    line.split(" = ")[0]: line.split(" = ")[1].strip("()").split(", ")
    for line in patterns[1].splitlines()
}

# Part 1
steps = 0
node = "AAA"
while node != "ZZZ":
    node = hops[node][instructions[steps % len(instructions)]]
    steps += 1
print(f"Part 1: {steps}")  # 19667

# Part 2
part1_nodes = [start for start in hops if start[-1] == "A"]
# Find the frequencies for the different starting points ending with 'A'
freqs = []
steps = 0
while len(freqs) < len(part1_nodes):
    for pos, node in enumerate(part1_nodes):
        next = hops[node][instructions[steps % len(instructions)]]
        if next[-1] == "Z":
            freqs.append(steps + 1)
        part1_nodes[pos] = next
    steps += 1
# Least Common Multiple is when all the frequencies match up
print(f"Part 2: {lcm(*freqs)}")  # 19185263738117

Part 1: 19667
Part 2: 19185263738117


In [ ]:
# Day 9: Mirage Maintenance
def differences(numbers):
    return [b - a for a, b in zip(numbers, numbers[1:])]


def predict(value):
    result = [value]
    while any(result[-1]):
        result.append(differences(result[-1]))
    return result


def part1(prediction):
    return sum(value[-1] for value in prediction)


def part2(prediction):
    # See problem description for details
    # A B X1 X2 X3
    #  C Y1 Y2 Y3
    # A = B - C
    c = 0
    for b in prediction[::-1]:
        a = b[0] - c
        c = a
    return a


sequence = data(9, ints)
print(f"Part 1: {sum(part1(predict(value)) for value in sequence)}")  # 1834108701
print(f"Part 2: {sum(part2(predict(value)) for value in sequence)}")  # 993

Part 1: 1834108701
Part 2: 993


In [ ]:
# Day 10: Pipe Maze
def new_dir(position, dir):
    right_turn = -1j
    left_turn = 1j
    # char, dir: turn
    # [-1, 1j, 1, -1j] = N, E, S, W
    turns = {
        ("F", -1): right_turn,
        ("F", -1j): left_turn,
        ("J", 1): right_turn,
        ("J", 1j): left_turn,
        ("7", 1j): right_turn,
        ("7", -1): left_turn,
        ("L", 1): left_turn,
        ("L", -1j): right_turn,
    }
    if (expanded_rows[position], dir) in turns:
        return dir * turns[(expanded_rows[position], dir)]
    return dir


def create_polygon(start_position, dir=-1):
    # Starting dir (-1 == North) comes from looking manually at the map
    position = start_position
    polygon = {start_position}
    while position != start_position or len(polygon) < 2:
        dir = new_dir(position, dir)
        position = position + dir
        polygon.add(position)

    return {(int(p.real), int(p.imag)) for p in polygon}


def create_intersections(polygon):
    "Create a grid with the number of intersections for each point based on a straight line from left to right. See https://en.wikipedia.org/wiki/Point_in_polygon"
    intersections = {}
    for row in range(max(polygon)[0]):
        current = 0
        last_bend = ""
        for col in range(max(polygon, key=lambda x: x[1])[1]):
            pos = row + 1j * col
            if (row, col) in polygon:
                if expanded_rows[pos] == "|":
                    current += 1
                # 7---L and J---F are equivalent to a straight vertical line
                if expanded_rows[pos] == "7" and last_bend == "L":
                    current += 1
                if expanded_rows[pos] == "J" and last_bend == "F":
                    current += 1
                if expanded_rows[pos] in "FJL7":
                    last_bend = expanded_rows[pos]
            else:
                intersections[(row, col)] = current
    return intersections


def points_inside_polygon(intersections):
    return sum(1 for p in intersections.values() if p % 2)


lines = data(10)
expanded_rows = {}  # (row, col): char
start_position = 0
for row, first_part in enumerate(lines):
    for col, char in enumerate(first_part):
        expanded_rows[(row + 1j * col)] = char
        if char == "S":
            start_position = row + 1j * col
            expanded_rows[(row + 1j * col)] = "J"  # Manually looking at map

polygon = create_polygon(start_position)
print(f"Part 1: {len(polygon) // 2}")  # 6733
print(f"Part 2: {points_inside_polygon(create_intersections(polygon))}")  # 435

Part 1: 6733
Part 2: 435


In [ ]:
# Day 11: Cosmic Expansion
def create_grid(lines):
    grid = {
        (row, col): char
        for row, line in enumerate(lines)
        for col, char in enumerate(line)
    }
    return grid


def shortest_paths(start, expansion):
    """
    Calculate shortest path from start to all possible positions
    """
    frontier = []
    heappush(frontier, (0, start))
    came_from = {}
    cost_so_far = defaultdict(lambda: inf)
    came_from[start] = None
    cost_so_far[start] = 0

    while frontier:
        current = heappop(frontier)[1]
        for next in part1_moves(current):
            new_cost = cost_so_far[current] + cost(current, next, expansion)
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost
                heappush(frontier, (priority, next))
                came_from[next] = current

    return cost_so_far


def cost(current, next, expansion):
    row, col = current
    to_row, to_col = next
    # Vertical movement
    if row - to_row and row_empty(row):
        return expansion
    # Horizontal movement
    if col - to_col and col_empty(col):
        return expansion
    return 1


@cache
def row_empty(row):
    cols = max(grid.keys(), key=lambda x: x[1])[1]
    return all(grid[(row, c)] == "." for c in range(cols + 1))


@cache
def col_empty(col):
    rows = max(grid.keys())[0]
    return all(grid[(r, col)] == "." for r in range(rows + 1))


@cache
def part1_moves(current):
    row, col = current
    candidates = {(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)}
    return {move for move in candidates if move in grid.keys()}


def total_cost(expansion=2):
    galaxies = {coord for coord, char in grid.items() if char == "#"}
    pairs = set(combinations(galaxies, 2))
    total_cost = 0
    for galaxy in galaxies:
        cost_to_reach = shortest_paths(galaxy, expansion)
        for (from_, to_) in pairs.copy():
            if galaxy == from_:
                total_cost += cost_to_reach[to_]
                pairs.remove((from_, to_))
    return total_cost


lines = data(11)
grid = create_grid(lines)
print(f"Part 1: {total_cost(expansion=2)}")  # 10165598, 25s
print(f"Part 2: {total_cost(expansion=1_000_000)}")  # 678728808158, 35s

Part 1: 10165598
Part 2: 678728808158


In [ ]:
# Day 12: Hot Springs
@cache
def arrangements(springs, damaged):
    # Initial input springs need to have a '.' at the end to make code below easier
    # Base cases
    if not damaged:
        if all(c in "?." for c in springs):
            return 1
        return 0
    if not springs:
        return 0

    damage = damaged[0]
    spring = springs[0]
    if spring == "?":
        if all(c in "?#" for c in springs[:damage]) and springs[damage] in "?.":
            # If we can get a full chunk of #:s and a stop, return that as one possibility
            # + the case where ? is a '.' as the other possibility
            return arrangements(springs[damage + 1 :], damaged[1:]) + arrangements(
                springs[1:], damaged
            )
        else:
            # We can't get a full chunk of '#', just return the case where ? is a '.'
            return arrangements(springs[1:], damaged)
    if spring == "#":
        # Need complete chunk + a pause
        if all(c in "?#" for c in springs[:damage]) and springs[damage] in "?.":
            return arrangements(springs[damage + 1 :], damaged[1:])
        else:
            return 0
    if spring == ".":
        return arrangements(springs[1:], damaged)


def unfold(springs, numbers):
    "Unfold springs and numbers, and add '.' to unfolded springs"
    return (springs + "?") * 4 + springs + ".", numbers * 5


lines = data(12)
part1, part2 = 0, 0
for first_part in lines:
    springs, damages = first_part.split()
    damages = tuple(ints(damages))
    part1 += arrangements(springs + ".", damages)
    part2 += arrangements(*unfold(springs, damages))

print(f"Part 1: {part1}")  # 7771
print(f"Part 2: {part2}")  # 10861030975833

Part 1: 7935
Part 2: 10887924681597


In [ ]:
# Day 13: Point of Incidence
def rows(grid):
    return max(grid, key=lambda coord: coord[0])[0] + 1


def cols(grid):
    return max(grid, key=lambda coord: coord[1])[1] + 1


def create_grid(chunk):
    return {
        (r, c): char
        for r, line in enumerate(chunk.splitlines())
        for c, char in enumerate(line)
    }


def first_item(items):
    if len(items):
        return first(items)
    else:
        return 0


def cols_to_the_left(grid):
    def row_is_mirrored(grid, row, mirror_column):
        first_part = [grid[(row, col)] for col in range(mirror_column)]
        second_part = [grid[(row, col)] for col in range(mirror_column, cols(grid))]
        return all(a == b for a, b in zip(first_part[::-1], second_part))

    def mirror_columns(grid, row):
        return {col for col in range(1, cols(grid)) if row_is_mirrored(grid, row, col)}

    set_list = [mirror_columns(grid, row) for row in range(rows(grid))]
    common_items = set.intersection(*set_list)
    return common_items


def rows_above(grid):
    def col_is_mirrored(grid, col, mirror_row):
        first_part = [grid[(row, col)] for row in range(mirror_row)]
        second_part = [grid[(row, col)] for row in range(mirror_row, rows(grid))]
        return all(a == b for a, b in zip(first_part[::-1], second_part))

    def mirror_rows(grid, col):
        return {row for row in range(1, rows(grid)) if col_is_mirrored(grid, col, row)}

    set_list = [mirror_rows(grid, col) for col in range(cols(grid))]
    common_items = set.intersection(*set_list)
    return common_items


def smudge(grid, row, col):
    "Modifies grid in place, make sure to restore"
    smudge = {"#": ".", ".": "#"}
    grid[(row, col)] = smudge[grid[(row, col)]]


def smudged_lines(grid, v, h):
    "Returns new values for vertical, horizontal"
    for row in range(rows(grid)):
        for col in range(cols(grid)):
            smudge(grid, row, col)
            if v_smudged := cols_to_the_left(grid):
                if v_smudged != v:
                    smudge(grid, row, col)
                    return v_smudged - v, set()

            if h_smudged := rows_above(grid):
                if h_smudged != h:
                    smudge(grid, row, col)
                    return set(), h_smudged - h
            smudge(grid, row, col)


patterns = data(13, sep="\n\n")
part1, part2 = 0, 0
for pattern in patterns:
    grid = create_grid(pattern)
    v = cols_to_the_left(grid)
    h = rows_above(grid)
    new_v, new_h = smudged_lines(grid, v, h)
    part1 += first_item(v) + first_item(h) * 100
    part2 += first_item(new_v) + first_item(new_h) * 100

print(f"Part 1: {part1}")  # 43614
print(f"Part 2: {part2}")  # 36771, 45s for both parts

Part 1: 43614
Part 2: 36771


In [ ]:
# Day 14: Parabolic Reflector Dish, optimized code with numpy
def create_grid(chunk):
    return np.array([list(line) for line in chunk.splitlines()], dtype=str)


def tilt_platform_east(platform):
    def tilt_line_east(line):
        # Move all O:s right up until the next #
        # Will add an extra '#' at the end, chop that off with [:-1]
        return (
            "".join(
                part.count(".") * "." + part.count("O") * "O" + "#"
                for part in line.split("#")
            )[:-1]
            + "\n"
        )

    result = ""
    for line in platform:
        line = "".join(line)
        result += tilt_line_east(line)
    return create_grid(result)


def spin_cycle(platform):
    platform = np.rot90(platform, -1)
    platform = tilt_platform_east(platform)  # North tilt
    platform = np.rot90(platform, -1)
    platform = tilt_platform_east(platform)  # West tilt
    platform = np.rot90(platform, -1)
    platform = tilt_platform_east(platform)  # South tilt
    platform = np.rot90(platform, -1)
    platform = tilt_platform_east(platform)  # East tilt

    return platform


def load(platform):
    result = 0
    for weight, row in enumerate(platform):
        result += (platform.shape[0] - weight) * np.count_nonzero(row == "O")
    return result


def load_at_iteration(start_of_sequence, sequence, iteration=1000000000):
    pos = (iteration - start_of_sequence) % len(sequence)
    return sequence[pos]


chunk = open("2023/14.txt").read().rstrip()

platform = create_grid(chunk)
platform = np.rot90(platform, -1)
platform = tilt_platform_east(platform)
print(f"Part 1: {load(np.rot90(platform, 1))}")  # 110565

platform = create_grid(chunk)
# Pattern emerges starting at cycle 102 and lasts until 143, then 144-185 is the same
# numbers repeated, etc. See day14-output.txt for output.
# for i in range(1, 144):
#     platform = spin_cycle(platform)
#     print(f"{i:>4.0f}: {load(platform)}")

# Here is the observed pattern:
# fmt: off
sequence = [ 89803, 89824, 89836, 89855, 89878, 89887, 89896, 89900, 89880, 89873, 89849, 89813, 89800, 89795, 89799, 89819, 89845, 89851, 89873, 89896, 89892, 89895, 89889, 89869, 89844, 89822, 89796, 89790, 89808, 89815, 89840, 89860, 89869, 89891, 89901, 89891, 89884, 89878, 89840, 89817, 89805, 89786 ]
# fmt: on
# And observed start of pattern:
current = 102

print(f"Part 2: {load_at_iteration(current, sequence, 1000000000)}")  # 89845

Part 1: 110565
Part 2: 89845


In [ ]:
# Day 15: Lens Library
def hash(value, char):
    return (value + ord(char)) * 17 % 256


def hash_string(string):
    value = 0
    for char in string:
        value = hash(value, char)
    return value


def box_index(box, label):
    for i, item in enumerate(box):
        if label == item[0]:
            return i
    return -1


def boxes(lenses):
    boxes = defaultdict(list)
    for lens in lenses:
        if lens[-1] == "-":
            label = lens[:-1]
            box = boxes[hash_string(label)]
            if (index := box_index(box, label)) != -1:
                # Found a box, remove it
                box.pop(index)
        else:
            label, value = lens.split("=")
            value = int(value)
            box = boxes[hash_string(label)]
            if (index := box_index(box, label)) != -1:
                # Found a box, replace the value
                box[index] = (label, value)
            else:
                box.append((label, value))
    return boxes


def focusing_power(boxes):
    result = 0
    for box in boxes:
        for slot, lens in enumerate(boxes[box], 1):
            result += (box + 1) * slot * lens[1]
    return result


lenses = data(15, sep=",")
print(f"Part 1: {sum(hash_string(string) for string in lenses)}")  # 510801
print(f"Part 2: {focusing_power(boxes(lenses))}")  # 212763

Part 1: 510801
Part 2: 212763


In [ ]:
# Day 16: The Floor Will Be Lava
def create_grid(lines):
    grid = {}
    for row, line in enumerate(lines):
        for col, char in enumerate(line):
            grid[row + 1j * col] = char
    return grid


def step_one_beam(grid, beam):
    "Return a list of new beams."
    # Directions: 1 = South, -1 = North, 1j = East, -1j = West
    pos, dir = beam
    # print(f"{grid[pos]} {pos} {dir=}")
    match grid[pos], dir:
        # Move one step in current dir, don't change dir
        case ".", _:
            return [(pos + dir, dir)]
        case "|", (-1 | 1):
            return [(pos + dir, dir)]
        case "-", (-1j | 1j):
            return [(pos + dir, dir)]
        # Mirror, horizontal movement
        case "/", (-1j | 1j):
            dir = dir * 1j
            return [(pos + dir, dir)]
        # Mirror, vertical movement
        case "/", (-1 | 1):
            dir = dir * -1j
            return [(pos + dir, dir)]
        # Mirror, horizontal movement
        case "\\", (-1j | 1j):
            dir = dir * -1j
            return [(pos + dir, dir)]
        # Mirror, vertical movement
        case "\\", (-1 | 1):
            dir = dir * 1j
            return [(pos + dir, dir)]
        # Split into two vertical beams
        case "|", (-1j | 1j):
            dir1, dir2 = -1, 1
            return [(pos + dir1, dir1), (pos + dir2, dir2)]
        # Split into two horizontal beams
        case "-", (-1 | 1):
            dir1, dir2 = -1j, 1j
            return [(pos + dir1, dir1), (pos + dir2, dir2)]
        case _:
            print(f"Unhandled:  {grid[pos]=} {dir=}")
            return (pos + dir, dir), None


def step_all_beams(grid, beams):
    new_beams = set()
    for beam in beams:
        new_beams.update(step_one_beam(grid, beam))
    # Only return beams that are inside grid
    return {beam for beam in new_beams if beam[0] in grid}


def energized(grid, start_beam=(0, 1j)):
    beams = {start_beam}
    all_visited = {start_beam[0]}
    visited = 1
    for i in range(10_000):
        beams = step_all_beams(grid, beams)
        all_visited.update({beam[0] for beam in beams})
        if i % 10 == 0:
            if visited == len(all_visited):
                return len(all_visited)
            visited = len(all_visited)
    return -1


def max_energized(grid):
    side_length = int(sqrt(len(grid)))  # Quadratic grid
    top_side = [(0 + col * 1j, 1) for col in range(side_length)]
    bottom_side = [(side_length - 1 + col * 1j, -1) for col in range(side_length)]
    left_side = [(row, 1j) for row in range(side_length)]
    right_side = [(row + (side_length - 1) * 1j, -1j) for row in range(side_length)]
    sides = top_side + bottom_side + left_side + right_side

    return max(energized(grid, beam) for beam in sides)


lines = r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....""".splitlines()

lines = data(16)
grid = create_grid(lines)
print(f"Part 1: {energized(grid)}")  # 6883, 1s
print(f"Part 2: {max_energized(grid)}") # 7228, 2m 16s

Part 1: 6883
Part 2: 7228


In [ ]:
# Day 17: Clumsy Crucible
def create_grid(lines):
    grid = {}
    for row, line in enumerate(lines):
        for col, number in enumerate(line):
            grid[row + 1j * col] = int(number)
    return grid


class Position(namedtuple("Position", ["pos", "dir", "straight_steps"])):
    def __lt__(self, other):
        # Complex numbers can't be put in heapq, but this makes it possible
        return abs(self.pos) < abs(other.pos)


def a_star(start, goal, moves):
    """
    A* search from state `start` to `goal`.
    Need to provide functions: `heuristic`, `cost`, and `moves`
    """
    frontier = []
    heappush(frontier, (0, start))
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[start] = 0

    while frontier:
        current = heappop(frontier)[1]
        if heuristic(current, goal) == 0:
            return cost_so_far[current]
        for next in moves(current):
            new_cost = cost_so_far[current] + cost(current, next)
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal)
                heappush(frontier, (priority, next))


def heuristic(current, goal) -> float:
    "Minimum estimation of cost to get from a to goal. Also used to determine when we have reached goal (heuristic returns 0)."
    return abs(current.pos - goal.pos)


def cost(current, next):
    result = 0
    if next.dir.real:
        for step in range(int(next.pos.real), int(current.pos.real), -next.dir):
            result += grid[(step + 1j * next.pos.imag)]

    if next.dir.imag:
        for step in range(
            int(next.pos.imag), int(current.pos.imag), int((1j * next.dir).real)
        ):
            result += grid[(next.pos.real + 1j * step)]

    return result


def moves1(current):
    possible_dirs = dirs - {-current.dir}  # can't turn back
    if current.straight_steps == 3:
        possible_dirs = possible_dirs - {current.dir}  # can't move straight
    for dir in possible_dirs:
        if current.pos + dir in grid:
            if dir == current.dir:
                yield Position(current.pos + dir, dir, current.straight_steps + 1)
            else:
                yield Position(current.pos + dir, dir, 1)


def moves2(current):
    possible_dirs = dirs - {-current.dir}  # can't turn back
    if current.straight_steps == 10:
        possible_dirs = possible_dirs - {current.dir}  # can't move straight
    for dir in possible_dirs:
        if dir == current.dir and (current.pos + dir) in grid:
            yield Position(current.pos + dir, dir, current.straight_steps + 1)
        elif (current.pos + 4 * dir) in grid:
            # Turn + move 4 blocks in new direction
            yield Position(current.pos + 4 * dir, dir, 4)


lines = data(17)

# grid and dirs are global
grid = create_grid(lines)
dirs = {1, -1, 1j, -1j}  # 1 = South, -1 = North, 1j = East, -1j = West

current = Position(0, 0, 0)  # pos, dir, straight_steps
goal_pos = max(grid.keys(), key=lambda x: x.real) + max(
    grid.keys(), key=lambda x: x.imag
)
goal = Position(goal_pos, 0, 0)
print(f"Part 1: {a_star(current, goal, moves1)}")  # 785, 3s
print(f"Part 2: {a_star(current, goal, moves2)}")  # 922, 6s

Part 1: 785
Part 2: 922


In [ ]:
# Day 18: Lavaduct Lagoon
def corners(lines, part2=False):
    position, rights_and_downs, points = 0, 0, []
    dirs1 = {"R": 1j, "L": -1j, "U": -1, "D": 1}
    dirs2 = {"0": 1j, "1": 1, "2": -1j, "3": -1}
    for line in lines:
        dir, steps, color = line.split()
        if part2:
            dir = color[-2]
            steps = int(color[2:-2], 16)
            dir = dirs2[dir]
        else:
            steps = int(steps)
            dir = dirs1[dir]
        position += steps * dir
        if dir in [1j, 1]:
            rights_and_downs += steps
        points.append(position)
    return points, rights_and_downs


def area(points, extra):
    # Shoelace formula: https://en.wikipedia.org/wiki/Shoelace_formula
    def shoelace(xs, ys):
        def shoelace_part(bases, diagonals):
            return sum(
                base * diagonals[(i + 1) % len(diagonals)]
                for i, base in enumerate(bases)
            )

        return (shoelace_part(xs, ys) - shoelace_part(ys, xs)) / 2

    def xs(points):
        return [int(p.imag) for p in points]

    def ys(points):
        return [int(p.real) for p in points]

    # Moving in a grid makes this different from a "normal" area. Need to account for all
    # points above (i.e. when moving right *or* left, not both), to the left (moving down
    # *or* up) + starting point in grid. See it as a padding above and to the left of the
    # polygon.
    return int(shoelace(xs(points), ys(points))) + extra + 1


lines = data("18")
points, extra = corners(lines)
print(f"Part 1: {area(points, extra)}")  # 92758
points, extra = corners(lines, part2=True)
print(f"Part 2: {area(points, extra)}")  # 62762509300678

Part 1: 92758
Part 2: 62762509300678


In [ ]:
# Day 19: Aplenty
Category = namedtuple("Category", ["name", "lower", "upper"])


def parse(chunks):
    workflows_input, parts_input = chunks
    workflows_input = workflows_input.splitlines()
    parts_input = parts_input.splitlines()
    parts = defaultdict(list)
    for part_input in parts_input:
        part_input = part_input.strip("{}")
        part = {}
        for category in part_input.split(","):
            label, value = category.split("=")
            value = int(value)
            part[label] = value
        parts["in"].append(part)

    workflows = {}
    for workflow_input in workflows_input:
        label, rules = workflow_input.split("{")
        rules = rules[:-1].split(",")
        workflows[label] = rules
    return parts, workflows


def parse_criteria(text):
    return text[0], text[1], int(text[2:])


def part_destination(part, rules):
    "Determine the new destination for `part` given `rules`."

    def process(category, value, rule):
        "Process a rule according to category, return either new workflow, A/R, or None"
        match rule.split(":"):
            case criteria, dest:
                rule_category, comparison, rule_value = parse_criteria(criteria)
                if rule_category == category:
                    if comparison == ">" and value > rule_value:
                        return dest
                    if comparison == "<" and value < rule_value:
                        return dest
                return None
            case dest:
                return first(dest)

    for rule in rules:
        for category, value in part.items():
            if dest := process(category, value, rule):
                return dest
    return None


def sort(parts, workflows):
    def sort_one(parts, workflows):
        "Perform one iteration of sorting, return new `parts`"
        new_parts = defaultdict(list)
        for source in parts:
            for part in parts[source]:
                if source in "AR":
                    new_parts[source].append(part)
                    continue
                dest = part_destination(part, workflows[source])
                new_parts[dest].append(part)
        return new_parts

    target = len(parts["in"])
    while len(parts["A"]) + len(parts["R"]) < target:
        parts = sort_one(parts, workflows)
    return parts


def ratings(parts):
    return sum(
        sum(part.values())
        for index, parts in parts.items()
        if index == "A"
        for part in parts
    )


def apply(rule, ranges):
    "Apply rule to ranges. Return current new ranges, destination, and destination ranges"
    current = []
    destination = []
    match rule.split(":"):
        case criteria, dest:
            name, comp, value = parse_criteria(criteria)
            for cat in ranges:
                dest_cat = cat
                current_cat = cat
                if cat.name == name and cat.lower < value < cat.upper:
                    # Split range
                    if comp == "<":
                        dest_cat = Category(cat.name, cat.lower, value - 1)
                        current_cat = Category(cat.name, value, cat.upper)
                    elif comp == ">":
                        dest_cat = Category(cat.name, value + 1, cat.upper)
                        current_cat = Category(cat.name, cat.lower, value)
                destination.append(dest_cat)
                current.append(current_cat)
            return current, dest, destination
        case dest:
            return [], first(dest), ranges


def accepted(applicable_ranges, workflows):
    "Calculate the ranges that will end up in `A`, i.e. accepted"
    accepted = []
    while applicable_ranges:
        new_ranges = {}
        for workflow in applicable_ranges:
            if workflow in "AR":
                continue
            ranges = applicable_ranges[workflow]
            for rule in workflows[workflow]:
                ranges, dest, destination = apply(rule, ranges)
                if dest == "A":
                    accepted.append(destination)
                new_ranges[dest] = destination
        applicable_ranges = new_ranges
    return accepted


def combinations(ranges):
    "How many combinations are possible with `ranges`"
    result = 1
    for cat in ranges:
        result *= (cat.upper + 1) - cat.lower
    return result


chunks = data(19, sep="\n\n")
parts, workflows = parse(chunks)
applicable_ranges = {"in": [Category(name, 1, 4000) for name in "xmas"]}

print(f"Part 1: {ratings(sort(parts, workflows))}")  # 449531
print(
    f"Part 2: {sum(combinations(a) for a in accepted(applicable_ranges, workflows))}"
)  # 122756210763577

Part 1: 449531
Part 2: 122756210763577


In [ ]:
# Day 20: Pulse Propagation
@dataclass
class Module:
    name: str
    type: str
    signal: bool = False  # False = low, True = high
    inputs: list = field(default_factory=lambda: [])
    outputs: list = field(default_factory=lambda: [])
    highs = 0
    lows = 0

    def pulse(self, signal):
        if signal:
            Module.highs += 1
        else:
            Module.lows += 1
        match self.type, signal:
            case "%", True:
                return
            case "%", False:
                self.signal = not self.signal
            case "&", _:
                self.signal = not all(m.signal for m in self.inputs)
            case "b", _:
                self.signal = False

    def connect(self, module):
        self.outputs.append(module)
        module.inputs.append(self)


def create_modules(lines):
    modules = {}
    for line in lines:
        source, targets = line.split(" -> ")
        type = source[0]
        name = source[1:]
        if source == "broadcaster":
            type = "b"
            name = source
        modules[name] = Module(name, type)

    # Connect all modules
    for line in lines:
        source, targets = line.split(" -> ")
        name = source[1:]
        if source == "broadcaster":
            name = source
        for target in targets.split(", "):
            if target not in modules:
                modules[target] = Module(name, "")
            modules[name].connect(modules[target])
    return modules


def part1(modules, repeats=1_000):
    for _ in range(repeats):
        to_pulse = [(modules["broadcaster"], False)]
        while to_pulse:
            module, signal = to_pulse.pop(0)
            module.pulse(signal)
            if not (module.type == "%" and signal):
                for target in module.outputs:
                    to_pulse.append((target, module.signal))
    return Module.highs * Module.lows


lines = data(20)
modules = create_modules(lines)
print(f"Part 1: {part1(modules)}")  # 1020211150

# Generate chart of how these registers are connected with mermaidchart, see 2023/20.svg.
# Looking at the registers and running some simulations such as:
# to_check = ["rz", "vm", "xn", "xg", "mz", "dd", "sp", "mj", "ms", "hx", "zg", "ls"]
# for i in range(1, 3_852):
#     press_button(modules)
# for name in to_check:
#     if modules[name].signal:
#         print("1", end=" ")
#     else:
#         print("0", end=" ")
# print(f" == {i}")

# you can see that there are four different counters. Looking at the 'nr' register which
# feed 'fn' you can see it depends on:

# kv xr qq dh nm rb xl jd bm fj pt lg
# Denoting arrow out to nr = 1, arrow in from nr = 0 we get:
#  1  1  0  1  1  1  0  1  1  1  1  1 == 4027

# Doing
# the same for the three other registers which feed 'nc' we get: lk, based on gk config
# fp cc tp hq ql bn rj jk dg mp kj fl
#  1  1  0  0  0  1  0  1  1  1  1  1 == 4003

# fh, based on gl 1 1 0 1 0 0 0 0 1 1 1 1 == 3851

# fn, based on nr 1 1 1 0 0 0 0 0 1 1 1 1 == 3847
#
# The answer is probably when these four frequencies intersect, or:
# print(f"Part 2: {lcm(4027, 4003, 3851, 3847)}")  # 238815727638557
# which is correct.

# After this analysis, I could implement:
def part2(modules, repeats=5_000):
    registers = ["lk", "fn", "fh", "hh"]
    values = []
    for i in range(1, repeats + 1):
        to_pulse = [(modules["broadcaster"], False)]
        while to_pulse:
            module, signal = to_pulse.pop(0)
            if module.name in registers and not signal:
                values.append(i)
            module.pulse(signal)
            if not (module.type == "%" and signal):
                for target in module.outputs:
                    to_pulse.append((target, module.signal))
    return lcm(*values)


modules = create_modules(lines)
print(f"Part 2: {part2(modules)}")  # 238815727638557

Part 1: 1020211150
Part 2: 238815727638557


In [ ]:
# Day 21: Step Counter
def create_grid(lines):
    grid = {
        (row, col): char
        for row, line in enumerate(lines)
        for col, char in enumerate(line)
    }
    return grid


def part1_moves(current):
    row, col = current
    candidates = {(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)}
    return {(r, c) for (r, c) in candidates if grid[(r % rows, c % cols)] in [".", "S"]}


def plots_reached(steps):
    # Reached plots toggle between on and off for each step
    if steps % 2:
        return sum(1 for dist in distance.values() if dist % 2 and dist <= steps)
    else:
        return sum(1 for dist in distance.values() if not dist % 2 and dist <= steps)


lines = data(21)
# Global variables
grid = create_grid(lines)
rows = max(grid.keys(), key=lambda x: x[0])[0] + 1
cols = max(grid.keys(), key=lambda x: x[1])[1] + 1

current = (rows // 2, cols // 2)  # 'S' is in the middle
frontier = []
frontier.append(current)
distance = {current: 0}

# Flood fill, 3_000_000 iterations. We need the distance dict to calculate number of plots
# you can reach in a certain number of steps
for _ in range(3_000_000):  # 12s
    current = frontier.pop(0)
    for next in part1_moves(current):
        if next not in distance:
            frontier.append(next)
            distance[next] = 1 + distance[current]

print(f"Part 1: {plots_reached(64)}")  # 3853


# Deduction / trial and error for part 2 is in git history, but removed here for brevity
# Chart this in excel
# steps = range(100, 500)

# last = 0
# cur = 0
# for step in steps:
#     cur = plots_reached(step)
#     print(f"{step}, {cur - last}")
#     last = cur

# and you see that [193, 324, 455] are part of the same series, the cycle time is the
# difference between two values
cycle = 324 - 193

# Our target is 26501365; 26501365 / 131 == 202300, but use one cycle less to assure we
# have reached a stable state
n = 26501365 // cycle - 1  # 202299
base = 26501365 - cycle * n  # Use value at 196 as a base point
base_value = plots_reached(base)  # Base value == 35259
# increase should be multiplied with i
increase = plots_reached(base + cycle) - plots_reached(base)  # 62548
# step_size should be multiplied with i(i-1) // 2
step_size = (plots_reached(base + 2 * cycle) - plots_reached(base + cycle)) - (
    plots_reached(base + cycle) - plots_reached(base)
)
# Check that this holds true for value at step 5
i = 5
assert base_value + i * increase + (i * (i - 1) // 2) * step_size == plots_reached(
    196 + i * cycle
)
# And finally:
i = n
# 639051580070841
print(f"Part 2: {base_value + i * increase + (i * (i - 1) // 2) * step_size}")

Part 1: 3853
Part 2: 639051580070841


In [ ]:
# Day 22: Sand Slabs
Point = namedtuple("Point", ["x", "y", "z"])


# Implementation of https://www.redblobgames.com/grids/line-drawing/ in 3D
@cache
def line(p0, p1):
    def lerp_point(p0, p1, t):
        def lerp(start, end, t):
            return start * (1 - t) + end * t

        return Point(lerp(p0.x, p1.x, t), lerp(p0.y, p1.y, t), lerp(p0.z, p1.z, t))

    def round_point(p):
        return Point(round(p.x), round(p.y), round(p.z))

    def diagonal_distance(p0, p1):
        dx = p1.x - p0.x
        dy = p1.y - p0.y
        dz = p1.z - p0.z
        return max(abs(dx), abs(dy), abs(dz))

    if p0 == p1:
        return {p0}
    points = set()
    N = diagonal_distance(p0, p1)
    for step in range(N + 1):
        t = step / N
        points.add(round_point(lerp_point(p0, p1, t)))
    return points


def overlap(b0, b1):
    "Returns true if bricks b0 and b1 overlaps in xy plane."
    start0, end0 = b0
    start1, end1 = b1
    line0 = line(Point(start0.x, start0.y, 0), Point(end0.x, end0.y, 0))
    line1 = line(Point(start1.x, start1.y, 0), Point(end1.x, end1.y, 0))
    return len(line0.intersection(line1)) > 0


def sorted_bricks(bricks, highest_first=False):
    return sorted(bricks, key=lambda p: min(p[0].z, p[1].z), reverse=highest_first)


def bricks_at_height(bricks, level):
    "Return all bricks at a certain level"
    return {brick for brick in bricks if level in (brick[0].z, brick[1].z)}


def drop_brick_one_step(at_rest, brick):
    "Tries to drop brick one step, returns bricks new coordinates. Returns same brick if it cannot be dropped further."
    start, end = brick
    from_height = min(start.z, end.z)
    if from_height > 1:
        if any(
            overlap(brick, resting)
            for resting in bricks_at_height(at_rest, from_height - 1)
        ):
            # print(f"{brick} overlaps and should stop at {from_height}")
            return brick
        else:
            # print(f"{brick} does not overlap and falls down to {from_height - 1}")
            start, end = brick
            return Point(start.x, start.y, start.z - 1), Point(end.x, end.y, end.z - 1)
    # print(f"{brick} at bottom")
    return brick


def drop_brick(brick, at_rest):
    new_brick = drop_brick_one_step(at_rest, brick)
    while new_brick != brick:
        brick = new_brick
        new_brick = drop_brick_one_step(at_rest, brick)
    return brick


def drop_all(bricks):
    "Drop all bricks as far as possible. Return bricks at rest as a frozenset."
    at_rest = []
    for brick in sorted_bricks(bricks):
        new_brick = drop_brick_one_step(at_rest, brick)
        prev = brick
        while new_brick != prev:
            prev = new_brick
            new_brick = drop_brick_one_step(at_rest, new_brick)
        # print(
        # f"Put {new_brick} to rest at height {min(new_brick[0].z, new_brick[1].z)}"
        # )
        at_rest.append(new_brick)
    return frozenset(at_rest)


@cache
def supports(brick, bricks):
    height = min(brick[0].z, brick[1].z)
    layer_below = bricks_at_height(bricks, height - 1)
    return {b for b in layer_below if overlap(b, brick) and b != brick}


@cache
def leans_on(brick, bricks):
    height = max(brick[0].z, brick[1].z)
    layer_above = bricks_at_height(bricks, height + 1)
    return {b for b in layer_above if overlap(b, brick) and b != brick}


def cannot_be_disintegrated(bricks):
    cannot_be_removed = set()
    for brick in bricks:
        supported_by = supports(brick, bricks)
        if len(supported_by) == 1:
            # print(f"{brick} only supported by {first(supported_by)}")
            cannot_be_removed.update(supported_by)
    return cannot_be_removed


def parse_bricks(lines):
    return [
        (Point(*ints(l.split("~")[0])), Point(*ints(l.split("~")[1]))) for l in lines
    ]


def will_fall(brick, bricks):
    "How many bricks will fall if brick is disintegrated"
    will_fall = set()
    to_knock_over = {brick}
    while to_knock_over:
        brick = to_knock_over.pop()
        will_fall.add(brick)
        might_fall_this_layer = leans_on(brick, bricks)
        for brick in might_fall_this_layer:
            if all(b in will_fall for b in supports(brick, bricks)):
                to_knock_over.add(brick)
    return len(will_fall) - 1


lines = """1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9""".splitlines()

# A brick is two Points, start and end of a line
lines = data(22)
bricks = drop_all(parse_bricks(lines))
supporting_beams = cannot_be_disintegrated(bricks)
print(f"Part 1: {len(bricks) - len(supporting_beams)}")  # 418, 16s
print(f"Part 2: {sum(will_fall(brick, bricks) for brick in bricks)}")  # 70702, instant

Part 1: 418
Part 2: 70702


In [3]:
# Day 23: A Long Walk - with NetworkX
def create_grid(lines):
    grid = {
        (row, col): char
        for row, line in enumerate(lines)
        for col, char in enumerate(line)
    }
    return grid


def part1_moves(current):
    row, col = current
    if grid[current] == "v":
        return [(row + 1, col)]
    if grid[current] == ">":
        return [(row, col + 1)]
    candidates = {(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)}
    result = set()
    for move in candidates:
        if move not in grid.keys():
            continue
        if grid[move] == "#":
            continue
        if move[0] < row and grid[move] == "v":
            continue
        if move[1] < col and grid[move] == ">":
            continue
        result.add(move)
    return result


def part2_moves(current):
    row, col = current
    candidates = {(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)}
    return {move for move in candidates if move in grid.keys() and grid[move] != "#"}


def part1_nodes():
    return {coord for coord, char in grid.items() if char in ">v"}


def part2_nodes():
    return {
        coord
        for coord in grid.keys()
        if grid[coord] == "." and all(grid[move] in "v>" for move in part2_moves(coord))
    }


def distance(current, target, nodes, moves):
    frontier = []
    frontier.append(current)
    distance = {current: 0}

    while frontier:
        current = frontier.pop(0)
        for next in moves(current):
            if next == target:
                return distance[current] + 1
            if next in nodes:
                # Don't move through nodes
                continue
            if next not in distance:
                frontier.append(next)
                distance[next] = 1 + distance[current]
    return 0


def locate_start_end():
    rows = max(grid.keys(), key=lambda x: x[0])[0] + 1
    cols = max(grid.keys(), key=lambda x: x[1])[1] + 1
    start = first([(0, col) for col in range(cols) if grid[(0, col)] == "."])
    end = first(
        [(rows - 1, col) for col in range(cols) if grid[(rows - 1, col)] == "."]
    )
    return start, end


def part1_graph():
    nodes = part1_nodes().union({start, end})
    DG = nx.DiGraph()
    for current, target in permutations(nodes, 2):
        if current in [target, end]:
            continue
        if dist := distance(current, target, nodes, part1_moves):
            # print(f"{current}->{target} is {dist} steps")
            DG.add_edge(current, target, weight=dist)
    return DG


def part2_graph():
    G = nx.Graph()
    nodes = part2_nodes().union({start, end})
    for current, target in permutations(nodes, 2):
        if dist := distance(current, target, nodes, part2_moves):
            G.add_edge(current, target, weight=dist)
    return G


def longest_path2(G):
    longest = 0
    for path in nx.all_simple_paths(G, start, end):
        if (w := nx.path_weight(G, path, weight="weight")) > longest:
            longest = w
    return longest


lines = data(23)
grid = create_grid(lines)
start, end = locate_start_end()
grid[start] = "v"
grid[end] = "v"

DG = part1_graph()
print(f"Part 1: {nx.dag_longest_path_length(DG)}")  # 2186, 6 seconds

G = part2_graph()
print(f"Part 2: {longest_path2(G)}")  # 6802, 2m25s

Part 1: 2186
Part 2: 6802


In [4]:
# Day 23: A Long Walk part 1 - not using networkx
# Reusing graph structure from networkx because I'm lazy. Could just as well have been a
# simple dict, might even speed things up a bit
def longest_path1(current, visited=frozenset()):
    if current == end:
        return 0

    best = 0
    visited = frozenset(visited.union({current}))
    for dest, w in DG[current].items():
        if dest in visited:
            continue
        new_cost = w["weight"] + longest_path1(dest, visited)
        if new_cost > best:
            best = new_cost
    return best


longest_path1(start)

2186

In [24]:
# Day 23: A Long Walk part 2, same approach is not working
# For some reason this gives 6833 and not 6802 as the answer. I don't understand why.
def longest_path2(current, visited=set()):
    if current == end:
        return 0

    best = 0
    visited = frozenset(visited.union({current}))
    for dest, w in G[current].items():
        if dest in visited:
            continue
        new_cost = w["weight"] + longest_path2(dest, visited)
        if new_cost > best:
            best = new_cost
    return best


G = part2_graph()
longest_path2(start)

6833

In [18]:
# Solution from https://www.reddit.com/r/adventofcode/comments/18oy4pc/comment/kelin3v/
def search(node, dist, best, stop=end, seen=set()):
    if node == stop:
        return dist
    if node in seen:
        return best

    seen.add(node)
    best = max(search(n, w["weight"] + dist, best) for n, w in G[node].items())
    seen.remove(node)

    return best


print(search(start, 0, 0)) # 6802, 1m45s

# And an optimization from the same thread
def dfs(location, weight_sum=0):
    global best
    if location == end:
        best = max(best, weight_sum)
    next_edges = G[location].items()
    for loc, w in next_edges:
        if loc in visited:
            continue
        visited.add(loc)
        dfs(loc, weight_sum + w["weight"])
        visited.remove(loc)


visited = set()
best = 0
dfs(start) # 1m10s
print(best) # 6802

6802


In [33]:
# Day 24: Never Tell Me The Odds
Hailstone = namedtuple("Hailstone", "x y z dx dy dz")


def intersects(h1, h2):
    "Returns the coordinate where h1 and h2 crosses in xy-plane"
    m1 = h1.dy / h1.dx
    m2 = h2.dy / h2.dx
    if m2 == m1:
        return inf, inf
    c1 = h1.y - m1 * h1.x
    c2 = h2.y - m2 * h2.x
    x = (c1 - c2) / (m2 - m1)
    y = m1 * x + c1
    # Make sure this happens in the future
    if ((x - h1.x > 0 and h1.dx > 0) or (x - h1.x < 0 and h1.dx < 0)) and (
        (x - h2.x > 0 and h2.dx > 0) or (x - h2.x < 0 and h2.dx < 0)
    ):
        return x, y
    return inf, inf


def inside(coord, test_area):
    x, y = coord
    lower, upper = test_area
    return (lower <= x <= upper) and (lower <= y <= upper)


def crossings(hailstones, test_area):
    return sum(
        1
        for h1, h2 in combinations(hailstones, 2)
        if inside(intersects(h1, h2), test_area)
    )


def part2(hailstones, coords_to_use=3):
    time = [Int(f"time_{i}") for i in range(coords_to_use)]
    x, y, z, dx, dy, dz = Ints("x y z dx dy dz")
    s = Solver()
    for i, hailstone in enumerate(hailstones[:coords_to_use]):
        nx, ny, nz, ndx, ndy, ndz = hailstone
        s.add(x + dx * time[i] == nx + ndx * time[i])
        s.add(y + dy * time[i] == ny + ndy * time[i])
        s.add(z + dz * time[i] == nz + ndz * time[i])
        s.add(time[i] >= 0)
    s.check()
    return s.model().eval(x + y + z)


lines = data(24)
test_area = (200000000000000, 400000000000000)
hailstones = [Hailstone(*ints(line)) for line in lines]
print(f"Part 1: {crossings(hailstones, test_area)}")  # 12783
print(f"Part 2: {part2(hailstones)}")  # 948485822969419, 2s

Part 1: 12783
Part 2: 948485822969419


In [94]:
# Day 25: Snowverload
lines = data(25)

G = nx.Graph()
for line in lines:
    src, dest = line.split(": ")
    for d in dest.split(" "):
        G.add_edge(src, d)

# I stumbled across k_edge_components when reading NetworkX documentation. Did some test
# on example data and ended up with something that works. I don't really understand the
# underlying algorithm... Time to look at other solutions on Reddit.
result = 1
for c in nx.k_edge_components(G, 4):
    result *= len(c)

print(f"Part 1: {result}")  # 506202, 1m30s

Part 1: 506202
